In [ ]:
# SG3 - FTN
# Topic: Hand-joints data preprocess & abstraction
# Done:  (1) Hand-joints data preprocess & abstraction
#        (2) Fuse left & right hand

In [ ]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import itertools 
import warnings
import math

In [ ]:
# Path
destination_folder = "D:/vr-project/.../data/result" #result
path_clean = "D:/vr-project/.../data/clean/SG3" #clean
clean_file = os.walk(path_clean) 

In [ ]:
# Create blank csv
df1 = pd.DataFrame(columns = range(7))
df1.columns = ["user_id", "game_id", "round_id", "device_id", "time_slot", "sample_size", "block_number"]
save_info_path = destination_folder + '/SG3_info_FTN_1s.csv'
df1.to_csv(save_info_path, index=False, header=True) #default: header=False

df2 = pd.DataFrame(columns = range(915)) # 5  + 5 * 7 * 26= 915
df2_str_list = []
for j in ["max", "min", "mean", "std", "med"]:
  for i in range(26):
    for k in ["BoneRotation.x", "BoneRotation.y", "BoneRotation.z", "BoneRotation.w", "BonePosition.x",  "BonePosition.y", "BonePosition.z"]:
      df2_str_list.append(k + "_" + str(i) + "_" + j)
df2_id_list = ["user_id", "game_id", "round_id", "device_id","block_id"]
df2_str_list = df2_id_list + df2_str_list 
df2.columns = df2_str_list
save_feat_path = destination_folder + '/SG3_ts_feature_FTN_1s.csv'   
df2.to_csv(save_feat_path, index=False, header=True)

In [ ]:
# (1) Divide block and abstract the information

for path, dir_list, file_list in clean_file:
  for file_name in file_list:

    print(file_name)
    filepath = os.path.join(path, file_name)
    raw = pd.read_csv(filepath)
    raw_data = raw.loc[:,"BoneRotation[0].x":"BonePosition[25].z"] # 182 features

    # get FBN_g 
    # FBN_g : how many blocks this ts need to be divided into
    str2 = file_name.split(".")
    str3 = str2[0].split("_")
    id_list = [int(element) for element in str3]
    g_id = int(id_list[1])

    # Build ts blocks
    TS = 2 # TS = 1 means around one second per block
    LEN = raw.shape[0]
    ts_delta = (int(raw.at[LEN-1, "Timestamp"]) - int(raw.at[0, "Timestamp"])) / pow(10,9)
    sample_rate = int(LEN/ts_delta) #floor
    granu = int(TS * sample_rate)
    df_wr = pd.DataFrame(columns = range(915))

    # Traverse Blocks in a ts
    for i in range(1, int(ts_delta/TS) + 1):
      
      #set block
      raw_block = raw_data[(i-1)*granu : (i*granu-1)]
      
      #compute statistics
      fea_block = [] # 5 statistics
      fea_block += list(raw_block.max())
      fea_block += list(raw_block.min())
      fea_block += list(raw_block.mean())
      fea_block += list(raw_block.std())
      fea_block += list(raw_block.median())
      df_block_fea = pd.DataFrame(fea_block)
      df_block_fea = df_block_fea.T
      
      #combine blocks feature
      str2 = file_name.split(".")
      str3 = str2[0].split("_")
      id_list = [int(element) for element in str3]
      id_list.append(i)
      
      id_df = pd.DataFrame(id_list)
      id_df = id_df.T
      id_df.columns = ["user_id", "game_id", "round_id", "device_id", "block_id"]
      df_block_row = pd.concat([id_df, df_block_fea], axis = 1, join = 'outer')
      df_wr = df_wr.append(df_block_row)
      

    # Save One file info
    str2 = file_name.split(".")
    str3 = str2[0].split("_")
    id_list = [int(element) for element in str3]

    id_list.append(ts_delta) # sec info: how many seconds
    id_list.append(LEN) # how many samples
    id_list.append(i+1) # how many blocks
    id_df = pd.DataFrame(id_list)
    id_df = id_df.T
    id_df.columns = ["user_id", "game_id", "round_id", "device_id", "time_slot", "sample_size", "block_number"]
    id_df.to_csv(save_info_path, mode='a', index=False, header=False)
    #time_slot: how long for user playing the game: Unit-second
    #sample_size:how many ts data
    #block_number:timeseries is divided into how many blocks
    
    # Post-process Patch
    df_wr.replace('-', 0, inplace=True)
    df_wr.fillna(0, inplace=True)
    df_wr = df_wr.drop(df_wr.columns[910:915], axis=1) #[）
    hd_ls = ["block_id", "device_id", "game_id", "round_id", "user_id"]
    for element in hd_ls:
        col_insert = df_wr.pop(element)
        df_wr.insert(0, element, col_insert)
    # Save file featues
    df_wr.to_csv(save_feat_path, mode='a', index=False, header=False)

In [ ]:
# (2) Fuse to CSV
feature_path =  destination_folder + '/SG3_ts_feature_FTN_1s.csv' 
raw = pd.read_csv(feature_path)
#
raw.isna().sum().sum()
df_5 = raw[(raw.device_id == 5)].drop(columns =["device_id"]) #left hand
df_7 = raw[(raw.device_id == 7)].drop(columns =["device_id"]) #right hand
df_merge = pd.merge(df_5,df_7,how = "outer", on = ["user_id", "game_id", "round_id", "block_id"])
#
print(df_merge.shape)
#suffix: x means left hand, y means right hand
#
df_merge = df_merge.replace('-', pd.NA)
df_merge = df_merge.fillna(0)
df_merge.isna().sum().sum()

In [ ]:
save_path = destination_folder + '/SG3_fuse_ts_feature_FTN_1s.csv'   
df_merge.to_csv(save_path, index=False, header=True)
print("SUCCESS SAVED File!")

In [ ]:
# Check result file size
feature_path =  destination_folder + '/SG3_fuse_ts_feature_FTN_1s.csv' 
raw = pd.read_csv(feature_path)
print(raw.shape)
print("READ SUCCESS")